## Data/CSV Preprocessing

In [24]:
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import os
from PIL import Image

In [2]:
from csv_Extractor_Rohan import Extractor 

In [3]:
path = "/Users/rohansujeesh/Desktop/DeepLearning/DL/cbis-ddsm/csv"
base_path = "/Users/rohansujeesh/Desktop/DeepLearning/DL/cbis-ddsm/"

In [4]:
test = Extractor(path)

In [5]:
test.get_file_names()

['mass_case_description_train_set.csv',
 'calc_case_description_test_set.csv',
 'cleaned_dicom_info.csv',
 'calc_case_description_train_set.csv',
 'meta.csv',
 'dicom_info.csv',
 'mass_case_description_test_set.csv']

In [6]:
test.load_df()

In [7]:
test.rename("mass_case_description_train_set.csv","mass_train")
test.rename("calc_case_description_train_set.csv","calc_train")

In [8]:
test.dataframes.keys()

dict_keys(['calc_case_description_test_set.csv', 'cleaned_dicom_info.csv', 'meta.csv', 'dicom_info.csv', 'mass_case_description_test_set.csv', 'mass_train', 'calc_train'])

In [9]:
test.preprocess_steps(filename=["calc_train","mass_train"],image_path=base_path)

In [10]:
test.modified_dataframe.keys()

dict_keys(['calc_train', 'mass_train'])

In [13]:
calc_train = test.modified_dataframe["calc_train"]

In [17]:
calc_train.head()


,patient_id,assessment,pathology,image file path,ROI mask file path,ROI image,full image
0,P_00005,3,MALIGNANT,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...
1,P_00005,3,MALIGNANT,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...
2,P_00007,4,BENIGN,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...
3,P_00007,4,BENIGN,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...
4,P_00008,2,BENIGN_WITHOUT_CALLBACK,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...,/Users/rohansujeesh/Desktop/DeepLearning/DL/cb...


In [19]:
print(len(calc_train))

1544


## Small Model Test

In [20]:
import model_Library_Rohan

In [21]:
model = model_Library_Rohan.base_unet_test()

2025-07-23 02:23:42.490401: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-07-23 02:23:42.490506: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-07-23 02:23:42.490872: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-07-23 02:23:42.491711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-23 02:23:42.492349: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [22]:
#create sample tensor for unet model feeding
n = 10
image = np.ones((n,512,512,1))
mask = np.ones((n,512,512,1))


In [ ]:
#Normalize and add grayscale channel, resize to unet model input
for i, path in enumerate (calc_train["full image"][:10]):
    im = Image.open(path)
    im=im.resize((512,512))
    im = np.asarray(im) / 255.0  
    im = np.expand_dims(im, axis=-1)  
    image[i]=im

In [27]:
for i, path in enumerate (calc_train["ROI image"][:10]):
    im = Image.open(path)
    im=im.resize((512,512))
    im = np.asarray(im) / 255.0  # normalize
    im = np.expand_dims(im, axis=-1)  # add channel
    mask[i]=im

In [28]:
train_set = (image,mask)

In [30]:
%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model.fit(train_set[0],train_set[1],batch_size=8,epochs=5)

Epoch 1/5


2025-07-23 02:26:48.494150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 14s 1s/step - loss: 0.6723 - accuracy: 0.8906
Epoch 2/5
2/2 [==============================] - 16s 1s/step - loss: 0.8076 - accuracy: 0.8942
Epoch 3/5
2/2 [==============================] - 15s 1s/step - loss: 0.4114 - accuracy: 0.8942
Epoch 4/5
2/2 [==============================] - 15s 1s/step - loss: 0.4948 - accuracy: 0.8942
Epoch 5/5
2/2 [==============================] - 12s 1s/step - loss: 0.4879 - accuracy: 0.8942
1min 12s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
